To compare various abstractions we'll use a dataset available from SciKit Learn library. The data is comprised of the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are 13 different measurements taken for different constituents found in the three types of wine. We will use various TF abstractions to classify the wine to one of the 3 possible labels

In [1]:
from sklearn.datasets import load_wine
wine_data = load_wine()
type(wine_data)

sklearn.utils.Bunch

sklearn.utils.Bunch object is very similar to a dictionary


In [2]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [3]:
print(wine_data.DESCR)

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

We will attempt to do a classification on this data.

In [4]:
feat_data = wine_data['data']
labels = wine_data['target']

# 1. Train Test Split
Because this dataset is small, we'll just do a simple 70/30 train test split and we won't have any holdout data set. 

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size=0.3, random_state=101)

# 2. Scale the Data

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Keep in mind we only fit the scaler to the training data, we don't want to assume we'll have knowledge of future test data. We just transform the test set, not fit.

In [8]:
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

# 3. Abstractions: KERAS:

In [9]:
import tensorflow as tf
from tensorflow.contrib.keras import models

/Users/nevinyilmaz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 3.1: Create the model:

In [10]:
dnn_keras_model = models.Sequential()
#Sequential: Linear stack of layers.
#it's called sequential because you basically have a sequence of layers. 

### 3.2 Add Layers to the model:

In [11]:
from tensorflow.contrib.keras import layers

In [12]:
#LAYER 1: INPUT
dnn_keras_model.add(layers.Dense(units=13, input_dim=13, activation='relu'))
#Just your regular densely-connected NN layer.
#after the first layer, you don't need to specif the size of the input anymore

#LAYER 2:
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))

#LAYER 3:
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))

#LAYER 4: OUTPUT
dnn_keras_model.add(layers.Dense(units=3, activation='softmax'))


### 3.3: Compile the model (make it ready for training):
Compile sets up all the layers as they should be for the training

In [14]:
from tensorflow.contrib.keras import losses, optimizers, metrics, activations

In [15]:
#losses.  : we can see that there are lots of loss functions, including mean squared error, binary crossentropy, categorical crossentropy etc.
#activations.  : all of the available activations


In [16]:
losses.sparse_categorical_crossentropy

<function tensorflow.python.keras._impl.keras.losses.sparse_categorical_crossentropy>

In [17]:
dnn_keras_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#loss='sparse_categorical_crossentropy': if your data is not one-hot encoded coming in we use this for Keras to understand
# if your data is already one-hot encoded then you can just use categorical crossentropy

### 3.4: Train the model:

In [18]:
dnn_keras_model.fit(scaled_x_train, y_train, epochs=50)

Epoch 1/50
124/124 [==============================]124/124 [==============================] - 0s 2ms/step - loss: 1.0998 - acc: 0.3548

Epoch 2/50
124/124 [==============================]124/124 [==============================] - 0s 373us/step - loss: 1.0857 - acc: 0.3952

Epoch 3/50
124/124 [==============================]124/124 [==============================] - 0s 212us/step - loss: 1.0725 - acc: 0.4274

Epoch 4/50
124/124 [==============================]124/124 [==============================] - 0s 136us/step - loss: 1.0594 - acc: 0.4597

Epoch 5/50
124/124 [==============================]124/124 [==============================] - 0s 95us/step - loss: 1.0463 - acc: 0.5323

Epoch 6/50
124/124 [==============================]124/124 [==============================] - 0s 173us/step - loss: 1.0333 - acc: 0.5484

Epoch 7/50
124/124 [==============================]124/124 [==============================] - 0s 134us/step - loss: 1.0189 - acc: 0.5887

Epoch 8/50
124/124 [=================

### 3.5 Predictions:

In [31]:
predictions = dnn_keras_model.predict_classes(scaled_x_test)
#...model.predict gives the softmax output: between 0 and 1
#...model.predict_classes gives the actual classes we want: 0,1,2

In [32]:
from sklearn.metrics import confusion_matrix, classification_report

In [33]:
print(classification_report(predictions, y_test))

             precision    recall  f1-score   support

          0       0.95      0.95      0.95        19
          1       0.86      0.95      0.90        20
          2       1.00      0.87      0.93        15

avg / total       0.93      0.93      0.93        54



In [34]:
predictions

array([0, 0, 2, 0, 2, 1, 2, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 2, 1, 1, 1, 2,
       2, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 1, 0, 0, 2, 1, 2, 1, 0, 0, 0,
       2, 1, 1, 2, 2, 1, 0, 1, 1, 0])